In [1]:
cd "h:\내 드라이브\Inventory_Order_update"

h:\내 드라이브\Inventory_Order_update


In [2]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd
import json

root_path=''

In [3]:
with open(root_path+'appdata/db_auth.json') as f:
        temp = json.load(f)
        server = temp['server']
        database = temp['database'] 
        username = temp['username'] 
        password = temp['password']

connection_string = 'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url)

In [4]:
query = f"""
SELECT 
    item.ItemLookupCode, 
    item.Quantity,
    sp.SupplierID, 
    dbo.Supplier.SupplierName, 
    dbo.Supplier.Code, 
    item.Description as Description,
    item.Price Price, 
    item.Cost itemCost, 
    sp.Cost spCost
FROM 
    dbo.Item item
LEFT JOIN 
    dbo.SupplierList sp ON item.ID=sp.ItemID
LEFT JOIN 
    dbo.Supplier ON sp.SupplierID=dbo.Supplier.ID
WHERE 
    item.DepartmentID IN (2,4,6)
    AND Item.Inactive = 0
ORDER BY 
    item.SupplierID,
    item.Description;
"""

with engine.connect() as conn, conn.begin():  
    fromPOS = pd.read_sql(query, conn, dtype={'ItemLookupCode': str})

In [5]:
all_amazon = pd.read_csv(root_path+'inv_data\Amazon_All+Listings+Report.txt', sep='\t')

all_amazon = all_amazon[['seller-sku', 'asin1', 'item-name', 'item-description',
'price', 'quantity', 'status', 'product-id']]

In [6]:
all_amazon

,seller-sku,asin1,item-name,item-description,price,quantity,status,product-id
0,AL_10 CLOSURE 4X4_280,B07SQG44ZQ,"Foxy Silver (Foxy Closure - 10"" HH Lace) - Unp...",<b>Alicia Beauty Foxy Silver Collections Brazi...,58.9,10,Active,660014727654
1,AL_10 CLOSURE 4X4_44,B07SQG9Y6H,"Foxy Silver (Foxy Closure - 10"" Hh Lace) - Unp...",<b>Alicia Beauty Foxy Silver Collections Brazi...,58.9,10,Active,660014727678
2,AL_10 CLOSURE 4X4_51,B07SXQYHFJ,"Foxy Silver (Foxy Closure - 10"" HH Lace) - Unp...",<b>Alicia Beauty Foxy Silver Collections Brazi...,58.9,10,Active,660014727685
3,AL_AMARA_2,B0849V3KW2,Foxy Lady (Amara - Synthetic Full Wig in 2,<b>Foxy Lady Beautiful Young Straight Short Le...,23.9,10,Active,660014731323
4,AL_AMARA_4,B0849YWDX8,Foxy Lady (Amara - Synthetic Full Wig in 4,<b>Foxy Lady Beautiful Young Straight Short Le...,23.9,10,Active,660014731330
...,...,...,...,...,...,...,...,...
43028,VF_YESICA_P,B086QNCSHN,"Vivica A. Fox High Heat Resistant Wigs, 26 Inc...",NaN,NaN,0,Incomplete,B086QNCSHN
43029,VF_YUCERA_P,B01B808WSC,Vivica A Fox Yucera Synthetic Fiber Natural Ba...,NaN,NaN,0,Incomplete,B01B808WSC
43030,VF_YUKI_P,B0C8ZQRNDS,Vivica A. Fox OXFORD - Remi Natural Human Hair...,NaN,NaN,0,Incomplete,B0C8ZQRNDS
43031,VF_ZINNIA_P,B09NBHCQDZ,VIVICA A FOX NATURAL HUMAN HAIR WIG - ZINNIA,NaN,NaN,0,Incomplete,B09NBHCQDZ


In [7]:
temp = all_amazon.merge(fromPOS[['ItemLookupCode', 'Price', 'itemCost', 'spCost']], how='left', left_on='product-id', right_on='ItemLookupCode')

In [8]:
temp2 = temp.fillna('')

In [9]:
temp3 = temp2[temp2.duplicated()]

In [10]:
check_list = temp[temp['price']<temp['Price']]

In [11]:
temp.to_excel('amazon_pirce_check.xlsx', index=False)